# Simple Neural Network
Based on I. Pointer chapter 2 and A. Verdone tutorial

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
import time
import pickle
import torch.cuda

## Data loader

In [23]:
# dataset class

class EEGdataset(Dataset):
    """My DL EEG dataset."""
    def __init__(self, tab_data):
        df = tab_data

        # Grouping variable names
        self.target = "condition"
        self.eeg_frame = df

        # Save target and predictors
        self.X = self.eeg_frame.drop(self.target, axis=1)
        self.y = self.eeg_frame[self.target]

    def __len__(self):
        return len(self.eeg_frame)

    def __getitem__(self, idx):
        # Convert idx from tensor to list due to pandas bug (that arises when using pytorch's random_split)
        if isinstance(idx, torch.Tensor):
            idx = idx.tolist()

        return [self.X.iloc[idx].values, self.y[idx]]

In [24]:
# load averaged data with eeg only
with open('.\\pickles\\data_ave.pkl', 'rb') as handle:
    ave = pickle.load(handle)
ave.shape

tab_data = ave.drop(['epoch', 'id'], axis = 1)
tab_data.shape

(7551, 95)

In [25]:
ave.groupby(['condition']).agg('count')['epoch']

condition
0    2207
1    2431
2    2913
Name: epoch, dtype: int64

In [26]:
# Load dataset
dataset = EEGdataset(tab_data)

# Split into training and test
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
trainset, testset = random_split(dataset, [train_size, test_size])

# Dataloaders
trainloader = DataLoader(trainset, batch_size = 160, shuffle = True)
validloader = DataLoader(testset, batch_size  = 40, shuffle = False)

## SimpleNet

In [27]:
# define network
class SimpleNet(nn.Module):

    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(94, 74)
        self.fc2 = nn.Linear(74, 60)
        self.fc3 = nn.Linear(60, 40)
        self.fc4 = nn.Linear(40,3)
    
    def forward(self, x):
        x = x.view(-1, 94)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

simplenet = SimpleNet()

# set Adam optimizer
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

# set loss function
loss_fn = torch.nn.CrossEntropyLoss()

In [28]:
# to GPU
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

simplenet.to(device)
device

device(type='cpu')

## Training 

Trains the model, copying batches to the GPU if required, calculating losses, optimizing the network and perform validation for each epoch.

In [29]:
def train(model, optimizer, loss_fn, trainloader, validloader, epochs=20, device="cpu"):
   
    start_time = time.time() # timer
    print('Device: {}'.format(device))
    if torch.cuda.is_available():
        print('GPU Model: {}'.format(torch.cuda.get_device_name(0)))
    
    train_loss_per_iter = []
    valid_loss_per_iter = []
    
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
#         running_loss = 0.0
        model.train()
        for batch in trainloader: # train loop
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs.float())
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
            
            # Save loss to plot
#             running_loss += loss.item()
            train_loss_per_iter.append(loss.item())
            
        training_loss /= len(trainloader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in validloader: # validate/test loop
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs.float())
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            
            valid_loss_per_iter.append(loss.item())
            
            correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(validloader.dataset)

        print('Epoch: {}, Device: {}, Training loss: {:.2f}, Validation loss: {:.2f}, Accuracy: {:.2f}'. \
              format(epoch, device, training_loss, valid_loss, num_correct / num_examples))
    print("\nTotal training time: %s seconds" % round(time.time() - start_time, 2))
    
    # Plot training loss curve
    %matplotlib widget
    plt.plot(np.arange(len(train_loss_per_iter)), train_loss_per_iter, "-", alpha=0.5, label="Training loss per epoch")
    plt.plot(np.arange(len(valid_loss_per_iter)), valid_loss_per_iter, "-", alpha=0.5, label="Validation loss per epoch")
#     plt.plot(np.arange(len(loss_per_iter), step=4) + 3, loss_per_batch, ".-", label="Loss per mini-batch")
    plt.xlabel("Number of epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

In [31]:
train(simplenet, optimizer, loss_fn, trainloader, validloader, epochs = 3, device = device)

Device: cpu


D:\Programy\Anaconda3\envs\mne\lib\site-packages\ipykernel_launcher.py:46: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 0, Device: cpu, Training loss: 0.90, Validation loss: 0.92, Accuracy: 0.58
Epoch: 1, Device: cpu, Training loss: 0.87, Validation loss: 0.88, Accuracy: 0.60
Epoch: 2, Device: cpu, Training loss: 0.86, Validation loss: 0.87, Accuracy: 0.61

Total training time: 4.29 seconds


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Making predictions

Labels are in alphanumeric order, so `cat` will be 0, `fish` will be 1. We'll need to transform the image and also make sure that the resulting tensor is copied to the appropriate device before applying our model to it.

In [10]:
labels = ['cat','fish']

img = Image.open("./val/fish/100_1422.JPG") 
img = img_transforms(img).to(device)


prediction = F.softmax(simplenet(img))
prediction = prediction.argmax()
print(labels[prediction]) 

NameError: name 'Image' is not defined

## Saving Models

We can either save the entire model using `save` or just the parameters using `state_dict`. Using the latter is normally preferable, as it allows you to reuse parameters even if the model's structure changes (or apply parameters from one model to another).

In [76]:
torch.save(simplenet, "/tmp/simplenet") 
simplenet = torch.load("/tmp/simplenet")    

PermissionError: [Errno 13] Permission denied: './tmp/simplenet'

In [74]:
cd D:\Bartek\JupyterLab Projects\Books related

D:\Bartek\JupyterLab Projects\Books related


In [69]:
torch.save(simplenet.state_dict(), "/tmp/simplenet")    
simplenet = SimpleNet()
simplenet_state_dict = torch.load("/tmp/simplenet")
simplenet.load_state_dict(simplenet_state_dict)   

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/simplenet'